In [1]:
import sys; sys.path.append("../")
from feature_extractor import FeatureExtractor
import torch
import numpy as np
import time
from cascade import CascadeClassifier

In [2]:
torch.cuda.is_available()

True

In [3]:
parent = '../hFeatures/'

# Features Extraction

In [4]:

feature_extractor = FeatureExtractor(shape=(19,19), 
                                     percentile=20,
                                     all_features_file=parent + "all_features.npz",
                                     selected_features_file=parent + "selected_features.npz",
                                     labels_file=parent + "labels.npy",
                                     indecies_file=parent + "indecies.npy",)

In [5]:

# feature_extractor.extractFeatures(pos_path='../new_data/train/face', 
#                                   neg_path='../new_data/train/non-face')
print('done')


done


In [6]:
# del feature_extractor.f2
# del feature_extractor.f3
# del feature_extractor.f4

In [7]:
indecies, X = feature_extractor.selectPercentile()
y = np.load(parent + 'labels.npy')
print(X.shape, y.shape)
print('done')


(12792, 10854) (10854,)
done


# Training

In [ ]:
s_t = time.time()
cascadeClassifier= CascadeClassifier(X, y, batchsize=500, verbose=True, layers=[200])
tr_acc = cascadeClassifier.train()

cascadeClassifier.save(parent + 'cascadeClassifier.pkl')

print('accuracy: ', tr_acc)
print('time taken: ', time.time() - s_t, 's')
print('done')

# Testing

In [9]:
# accuracy imports
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [11]:
s_t = time.time()
t_f_idx_map, t_features, t_labels = feature_extractor.extractFeaturesByIndecies(pos_path='../new_data/test/face', 
                                            neg_path='../new_data/test/non-face',
                                            cascadeClassifier=cascadeClassifier)
print(t_features.shape, t_labels.shape)
predictions = cascadeClassifier.predict(t_features, t_f_idx_map)
print('test accuracy: ', np.sum(predictions == t_labels) / t_labels.shape[0])
print('time taken: ', time.time() - s_t, 's')

Now reading dataset...
Now extracting features from dataset...
(185, 2715) (2715,)
test accuracy:  0.9771639042357274
time taken:  5.589809894561768 s


In [12]:
# report
print('accuracy: ', accuracy_score(t_labels, predictions))
print('f1 score: ', f1_score(t_labels, predictions))
print('precision: ', precision_score(t_labels, predictions))
print('recall: ', recall_score(t_labels, predictions))
print('confusion matrix: \n', confusion_matrix(t_labels, predictions))
print('classification report: \n', classification_report(t_labels, predictions))


accuracy:  0.9771639042357274
f1 score:  0.9658213891951488
precision:  0.9636963696369637
recall:  0.9679558011049724
confusion matrix: 
 [[1777   33]
 [  29  876]]
classification report: 
               precision    recall  f1-score   support

           0       0.98      0.98      0.98      1810
           1       0.96      0.97      0.97       905

    accuracy                           0.98      2715
   macro avg       0.97      0.97      0.97      2715
weighted avg       0.98      0.98      0.98      2715

